In [1]:
import sys
sys.path.append('..')

In [52]:
import torch

# create a  lower triangular matrix 
n =  5
mask = torch.tril(torch.ones(n, n))
weight = torch.rand(n, n)
# create a permutation matrix of size n x n
all_perms = []
for _ in range(10):
    perm = torch.zeros((n, n))
    perm[torch.arange(n), torch.randperm(n)] =  1
    all_perms.append(perm)
P = torch.stack(all_perms)
# print(P)

# permute the mask using the permutation matrix
P_inv = P.transpose(1, 2)
mask = P @ mask.repeat(10, 1, 1) @ P_inv

# print(weight)
# print(mask * weight)
x = torch.rand(10, n)
bias = torch.rand(n)
# print("x = ", x)
# print("bias = ", bias)
# print("mask = ", mask)
# print(x[None, :])
print((mask * weight)[0] @ x[0] + bias)
# print(x[:, :, None])
print(torch.bmm((mask * weight), x[:, :, None]).squeeze() + bias)
# print((mask * weight) @ x)
print(((mask * weight) @ x.T + bias[:, None]).transpose(1, 2)[0])

print((mask[0,:,:] * weight) @ x[0, :].T + bias)

tensor([0.9167, 0.7454, 0.7233, 0.3633, 1.2489])
tensor([[0.9167, 0.7454, 0.7233, 0.3633, 1.2489],
        [0.6330, 0.8856, 0.3111, 0.7023, 0.8895],
        [1.0559, 1.2179, 0.3244, 0.4147, 0.9810],
        [1.9729, 1.2073, 1.6316, 0.5023, 1.5721],
        [0.7000, 2.0285, 2.5598, 0.3901, 1.2406],
        [1.1399, 1.7895, 2.6397, 1.3312, 0.7102],
        [1.2733, 1.1020, 0.9281, 0.8327, 1.7330],
        [0.7740, 1.8661, 1.5178, 0.7077, 1.4917],
        [1.4559, 1.0026, 1.0521, 1.1471, 0.6872],
        [1.6537, 1.3633, 1.3486, 1.4246, 0.6920]])
tensor([[0.9167, 0.7454, 0.7233, 0.3633, 1.2489],
        [0.6365, 1.0140, 0.7080, 0.3677, 0.9087],
        [0.7536, 1.2165, 0.8818, 0.3701, 1.1156],
        [1.4021, 1.7697, 1.6316, 0.3943, 1.9966],
        [1.1964, 1.4657, 1.3826, 0.3901, 1.6937],
        [1.0231, 1.7824, 1.5434, 0.3916, 1.6141],
        [1.2823, 1.4363, 1.2928, 0.3803, 1.7330],
        [1.0457, 1.5934, 1.2621, 0.3875, 1.4917],
        [0.9224, 1.0360, 0.7995, 0.3746, 1.3252],


In [1]:
import lightning
import lightning.pytorch.callbacks
from ocd.training import OrderedTrainingModule


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2
# set callbacks for the trainer
callbacks = [
    # monitor the learning rate (log to tensorboard)
    lightning.pytorch.callbacks.LearningRateMonitor(logging_interval="epoch"),
]

trainer = lightning.Trainer(
    # accelerator="mps",  # remove this line to run on CPU
    callbacks=callbacks,
    # precision=16, # for mixed precision training
    # gradient_clip_val=1.0,
    # gradient_clip_algorithm="value",
    max_epochs=10000,
    track_grad_norm="inf",
    log_every_n_steps=1,
    # overfit_batches=3,
    # detect_anomaly=True,
)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [4]:
# setup data
from ocd.data import CausalDataModule
import dycode
import torch

dycode.register_context(torch)

dm = CausalDataModule(
    name="asia",  # small dataset asia
    observation_size=4096,  # number of observation samples
    intervention_size=256,  # set to 0 for no intervention
    batch_size=128,
    num_workers=0,  # set to 0 for no multiprocessing
    val_size=0,  # 10% of data for validation, or use int for exact number of samples, set to 0 for no validation
    pin_memory=True,  # set to True for faster data transfer to GPU (if available)
)
dm.setup("fit")


[bnlearn] >Import <asia>
[bnlearn] >Loading bif file </home/hamidreza/Work/myprojects/ocd/venv/lib/python3.10/site-packages/bnlearn/data/asia.bif>
[bnlearn] >Check whether CPDs sum up to one.
[bnlearn] >Check whether CPDs associated with the nodes are consistent: True


In [5]:
# set torch.anomaly_detection(True) to debug
import torch

# Extract the category sizes
in_features = dm.train_data[0].features_values


# torch.autograd.set_detect_anomaly(True)
tm = OrderedTrainingModule(
    in_covariate_features=in_features,
    hidden_features_per_covariate=[
        [128 for i in range(len(in_features))],
        [64 for i in range(len(in_features))],
        [32 for i in range(len(in_features))],
    ],
    bias=False,
    batch_norm=False,
    criterion_args=dict(
        terms=[
            "ocd.training.terms.OrderedLikelihoodTerm",
            dict(
                name="norm(gamma)",
                term_function='lambda training_module: training_module.model.Gamma.norm(float("inf"))',
                factor=0,
            ),
            dict(
                name="norm(layers)",
                term_function='lambda training_module: max([layer.linear.weight.norm(float("inf")) for layer in training_module.model.made.layers])',
                factor=0,
            )
            # dict(
            #     name='nothing',
            #     term_function='def term(training_module, batch):\n\ttraining_module.batch=batch\n\treturn torch.zeros(1, device=batch.device)',
            #     factor=0,
            # )
        ]
    ),
    optimizer=['torch.optim.Adam', 'torch.optim.Adam'],
    optimizer_parameters=['model.made', 'model.Gamma'],
    optimizer_args=[
        dict(
            weight_decay=0.0001,
        ),
        dict()
    ],
    optimizer_is_active=[
        'lambda training_module: training_module.current_epoch % 10 < 10',
        'lambda training_module: training_module.current_epoch % 10 < 10',
    ],
    tau_scheduler="lambda training_module: max(0.003, 0.5 * 0.98 ** (training_module.current_epoch // 1))",
    n_sinkhorn_scheduler="lambda training_module: min(60, max(20, 20 + ((training_module.current_epoch - 20) // 10)))",
    lr=0.001,
    scheduler="torch.optim.lr_scheduler.ExponentialLR",
    scheduler_interval="epoch",
    scheduler_args={"gamma": 0.999},
)


In [10]:
trainer.fit(tm, dm)


[bnlearn] >Import <asia>
[bnlearn] >Loading bif file </home/hamidreza/Work/myprojects/ocd/venv/lib/python3.10/site-packages/bnlearn/data/asia.bif>
[bnlearn] >Check whether CPDs sum up to one.
[bnlearn] >Check whether CPDs associated with the nodes are consistent: True



  | Name  | Type                   | Params
-------------------------------------------------
0 | model | SinkhornOrderDiscovery | 675 K 
-------------------------------------------------
675 K     Trainable params
0         Non-trainable params
675 K     Total params
2.704     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 20it [00:00, ?it/s]

In [9]:
print(dm.trainer.datamodule.datasets[2].dag)

[[0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
